# Bluetooth Arm test

This code will test the arm over bluetooth, and also test kinematics on the arm.

In [ ]:
%load_ext autoreload
%autoreload 2
%aimport dynamixel

In [ ]:
from arm import Arm, JointPathWaypoint
import kinematics
import time
import numpy as np
from util import SE3

In [ ]:
PORT = '/dev/tty.borgla_test'
BAUD = 9600
WRITE_PACKET_DELAY = 0.15  # This is because bluetooth is a bit slow
# PORT = '/dev/tty.usbmodem1101'
# BAUD = 9600
# WRITE_PACKET_DELAY = 0.01

Robot = lambda: Arm(PORT, BAUD, write_packet_delay=WRITE_PACKET_DELAY)

In [ ]:
# Test connection to all servos by pinging
# We expect print 6 lines, all with error=0.  See https://emanual.robotis.com/docs/en/dxl/protocol1/#error
with Robot() as robot:
    robot.ping(0)
    robot.ping(1)
    robot.ping(2)
    robot.ping(3)
    robot.ping(4)
    robot.ping(5)
    for msg in robot.read_all_msgs():
        print(msg)

In [ ]:
# Allow hand-positioning of arm and print joint angles
with Robot() as robot:
    robot.disable_all()
    while input() != 'q':
        print(robot.read_all_joint_angles_deg())

In [ ]:
# Check FK by disabling motors and printing tip position as the robot is moved by hand
with Robot() as robot:
    robot.disable_all()
    print(f'{"Tip Position (m)":25}\t{"Joint Angles (deg)":30}')
    while True:
        try:
            q = np.array(robot.joint_angles_deg())
            wTtip, _ = kinematics.fk_deg(q)
            wPtip = wTtip[:3, 3]
            print(f'{str(wPtip.round(3)):25}\t{str(q.round(0)):30}', end='\r')
        except KeyboardInterrupt:
            break
    print()
    print(wTtip)

In [ ]:
# Set arm to 0 position
with Robot() as robot:
    robot.enable_all()
    robot.go_to_blocking([0, 0, 0, 0, 0])

In [ ]:
# Test following a path (dipping paint)
prep, hover, dip_pos, rub1, rub2 = (
    [20.087976539589448, -23.900293255131956, 83.72434017595307, 3.6656891495601087, 100.73313782991202],
    [22.727272727272748, -28.00586510263929, 106.30498533724341, 3.3724340175953103, 86.95014662756597],
    [22.140762463343123, -23.900293255131956, 115.10263929618768, 3.3724340175953103, 79.61876832844574],
    [23.020527859237546, -26.53958944281524, 114.51612903225806, 3.3724340175953103, 85.77712609970675],
    [22.140762463343123, -12.756598240469202, 104.83870967741936, 3.6656891495601087, 75.51319648093842],)

def dip():
    path = [
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),            # home
        JointPathWaypoint(q=prep, tol=5, timeout=None, pause=0),      # lower
        JointPathWaypoint(q=hover, tol=5, timeout=None, pause=0), # hover paint
        JointPathWaypoint(q=dip_pos, tol=5, timeout=None, pause=0), # dip
        JointPathWaypoint(q=rub1, tol=5, timeout=None, pause=0), # rub
        JointPathWaypoint(q=rub2, tol=5, timeout=None, pause=0), # rub
        JointPathWaypoint(q=hover, tol=5, timeout=None, pause=0), # lift paint
        JointPathWaypoint(q=prep, tol=5, timeout=None, pause=0),      # raise
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),            # home
    ]
    with Robot() as robot:
        robot.enable_all()
        robot.set_speed(50)
        robot.execute_joint_path(path, verbosity=0, default_speed=50)

In [ ]:
with Robot() as robot:
    robot.disable_all()
    time.sleep(0.5)

In [ ]:
# CENTER_POINT = np.array([-0.03205, 0.165, 0.24525])
CENTER_POINT = np.array([-0.03205, 0.147, 0.2493])
MAX_VEL = 50

In [ ]:
ok, q = kinematics.ik_deg(SE3.P(CENTER_POINT) @ SE3.RotX(-np.pi / 2), elbow_mode='neg')
assert ok, 'IK failed'
print(q)
with Robot() as robot:
    robot.go_to_blocking(q)

In [ ]:
def create_path(centerPxyz, vmax=MAX_VEL, angle=-np.pi / 2, elbow_mode='neg'):
    # First, add the ease in/out
    ease_point = np.array([0, -0.1, 0.0])
    centerPxyz = np.vstack([ease_point + centerPxyz[0], centerPxyz, ease_point + centerPxyz[-1]])
    # Now turn positions into global poses
    wTxyz = [(SE3.P(p + CENTER_POINT) @ SE3.RotX(angle)) for p in centerPxyz]
    # Finally, convert to joint angles
    qs = [kinematics.ik_deg(T, elbow_mode=elbow_mode) for T in wTxyz]
    assert all(q[0] for q in qs), f'IK failed: {[q[0] for q in qs]}'
    qs = [q[1] for q in qs]
    assert all(q[-1] > -108 for q in qs), f'Joint limit reached: \n{np.array(qs)}'
    # Now, handle velocities
    qdots = [kinematics.joint_vels_deg(q1, q2, max_vel=vmax) for q1, q2 in zip(qs[:-1], qs[1:])]
    qdots.insert(0, [vmax, 0.8*vmax, vmax, vmax, 0.8*vmax])
    qdots = [np.clip(np.abs(qdot), vmax * 0.15, None) for qdot in qdots]
    # Finally, create and return the path
    return [
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
        *[JointPathWaypoint(q=q, speeds=qdot, tol=5, timeout=None, pause=0) for q, qdot in zip(qs, qdots)],  # trajectory
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
    ]

In [ ]:
# Move to painting position
TO_PAINT_PATH = create_path([[0, 0.05, 0]])

def prep_to_paint():
    with Robot() as robot:
        robot.enable_all()
        robot.set_speed(50)
        robot.set_compliance_slopes(16)
        robot.execute_joint_path(TO_PAINT_PATH[:2], verbosity=0, default_speed=50)
def start_paint():
    with Robot() as robot:
        robot.enable_all()
        robot.set_speed(50)
        robot.set_compliance_slopes(16)
        robot.execute_joint_path(TO_PAINT_PATH[2:-2], verbosity=0, default_speed=50)
def stop_paint():
    with Robot() as robot:
        robot.enable_all()
        robot.set_speed(50)
        robot.set_compliance_slopes(16)
        robot.execute_joint_path(TO_PAINT_PATH[-2:-1], verbosity=0, default_speed=50)
def move_to_home():
    with Robot() as robot:
        robot.enable_all()
        robot.set_speed(50)
        robot.set_compliance_slopes(16)
        robot.execute_joint_path(TO_PAINT_PATH[-1:], verbosity=0, default_speed=50)

In [ ]:
dip()

In [ ]:
prep_to_paint()

In [ ]:
start_paint()
input()
stop_paint()

In [ ]:
move_to_home()

In [ ]:
t = np.linspace(0, 2 * np.pi, 9)[:, None]
positions = np.hstack([np.cos(t), 0 * t, np.sin(t)]) * 0.03
path = create_path(positions, vmax=75)

with Robot() as robot:
    robot.enable_all()
    robot.set_speed(75)
    robot.set_compliance_slopes(16)
    robot.execute_joint_path(path, verbosity=2, default_speed=75)

In [ ]:
path = create_path(np.array([0, -0.05, 0.1])[None, :], vmax=75)

with Robot() as robot:
    robot.enable_all()
    robot.set_speed(75)
    robot.set_compliance_slopes(32)
    robot.execute_joint_path(path[:3], verbosity=2, default_speed=75)
    input()
    robot.execute_joint_path(path[3:], verbosity=2, default_speed=75)

In [ ]:
# Try tilting the brush
Ts = [(SE3.P(CENTER_POINT + [0, 0, 0.1]) @ SE3.RotX(-np.pi / 2 + angle)) for angle in np.linspace(0, .5, 5)]
qs = [kinematics.ik_deg(T, elbow_mode='pos') for T in Ts]
assert all(q[0] for q in qs), f'IK failed: {[q[0] for q in qs]}'
qs = [q[1] for q in qs]
assert all(q[-1] > -108 for q in qs), f'Joint limit reached: \n{np.array(qs)}'

# qdots = [kinematics.joint_vels_deg(q1, q2, max_vel=MAX_VEL) for q1, q2 in zip(qs[:-1], qs[1:])]
# qdots.insert(0, [MAX_VEL, 0.8*MAX_VEL, MAX_VEL, MAX_VEL, 0.8*MAX_VEL])
# qdots = [np.clip(np.abs(qdot), 5, None) for qdot in qdots]

path = [
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
        # *[JointPathWaypoint(q=q, speeds=qdot, tol=5, timeout=None, pause=0) for q, qdot in zip(qs, qdots)],  # trajectory
        *[JointPathWaypoint(q=q, tol=5, timeout=None, pause=0) for q in qs],  # trajectory
        JointPathWaypoint(q=[0, 0, 0, 0, 0], tol=5, timeout=None, pause=0),  # home
    ]

with Robot() as robot:
    robot.enable_all()
    robot.set_compliance_slopes(16)
    robot.execute_joint_path(path, verbosity=2, default_speed=MAX_VEL)